In [87]:
# First we import all the libaries we're going to use

from bs4 import BeautifulSoup
import requests
import pandas as pd

In [88]:
# We get the URL of the Wikipedia page

url = "https://en.wikipedia.org/wiki/List_of_largest_companies_by_revenue"

# Send a GET request to the URL

page = requests.get(url)

# Parse the HTML content of the page

soup = BeautifulSoup(page.text, 'html.parser')


In [89]:
# Here we try to find the table with the desired data

soup.find_all('table')

[<table class="wikitable sortable sticky-header-multi sort-under" style="text-align:left;">
 <tbody><tr>
 <th rowspan="2" scope="col">Rank
 </th>
 <th rowspan="2" scope="col">Name
 </th>
 <th rowspan="2" scope="col">Industry
 </th>
 <th scope="col">Revenue
 </th>
 <th scope="col">Profit
 </th>
 <th rowspan="2" scope="col">Employees
 </th>
 <th rowspan="2" scope="col">Headquarters<sup class="reference" id="cite_ref-4"><a href="#cite_note-4">[note 1]</a></sup>
 </th>
 <th rowspan="2" scope="col"><a href="/wiki/State-owned_enterprise" title="State-owned enterprise">State-owned</a>
 </th>
 <th class="unsortable" rowspan="2" scope="col"><abbr title="Reference(s)">Ref.</abbr>
 </th>
 <th rowspan="2" scope="col">Revenue per worker
 </th></tr>
 <tr>
 <th colspan="2" scope="col"><small>USD millions</small>
 </th></tr>
 <tr>
 <th scope="col">1
 </th>
 <td><a href="/wiki/Walmart" title="Walmart">Walmart</a></td>
 <td><a href="/wiki/Retail" title="Retail">Retail</a></td>
 <td style="text-align:cen

In [90]:
# Since there are multiple tables in this site, one way to specify the table we want is by adding the class name

table = soup.find('table', class_ = 'wikitable sortable sticky-header-multi sort-under')
print(table)

<table class="wikitable sortable sticky-header-multi sort-under" style="text-align:left;">
<tbody><tr>
<th rowspan="2" scope="col">Rank
</th>
<th rowspan="2" scope="col">Name
</th>
<th rowspan="2" scope="col">Industry
</th>
<th scope="col">Revenue
</th>
<th scope="col">Profit
</th>
<th rowspan="2" scope="col">Employees
</th>
<th rowspan="2" scope="col">Headquarters<sup class="reference" id="cite_ref-4"><a href="#cite_note-4">[note 1]</a></sup>
</th>
<th rowspan="2" scope="col"><a href="/wiki/State-owned_enterprise" title="State-owned enterprise">State-owned</a>
</th>
<th class="unsortable" rowspan="2" scope="col"><abbr title="Reference(s)">Ref.</abbr>
</th>
<th rowspan="2" scope="col">Revenue per worker
</th></tr>
<tr>
<th colspan="2" scope="col"><small>USD millions</small>
</th></tr>
<tr>
<th scope="col">1
</th>
<td><a href="/wiki/Walmart" title="Walmart">Walmart</a></td>
<td><a href="/wiki/Retail" title="Retail">Retail</a></td>
<td style="text-align:center;"><span typeof="mw:File"><s

In [91]:
# We need to have the name of each Column titles or header so we make a list that finds them in using a FOR Loop

headers = []
for th in table.find_all('th'):
    headers.append(th.text.strip())
    
print(headers)

['Rank', 'Name', 'Industry', 'Revenue', 'Profit', 'Employees', 'Headquarters[note 1]', 'State-owned', 'Ref.', 'Revenue per worker', 'USD millions', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50']


In [92]:
# We need to skip the headers that are after USD Millions because those are the ranks

headers = []
for th in table.find_all('th')[:10]:
    headers.append(th.text.strip())
    
print(headers)

['Rank', 'Name', 'Industry', 'Revenue', 'Profit', 'Employees', 'Headquarters[note 1]', 'State-owned', 'Ref.', 'Revenue per worker']


In [93]:
# Next we extract the data rows from the table and put each row into it's own list
rows = []
for tr in table.find_all('tr')[2:]:  # Skip the header row
    cells = tr.find_all('td')
    row = [cell.text.strip() for cell in cells]
    rows.append(row)

print(rows)

[['Walmart', 'Retail', '$611,289', '$11,680', '2,100,000', 'United States', '', '[1]', '$291,090.00'], ['Saudi Aramco', 'Oil and gas', '$603,651', '$159,069', '70,496', 'Saudi Arabia', '', '[4]', '$8,562,911.37'], ['Amazon', 'Retail', '$574,785', '$30,425', '1,525,000', 'United States', '', '[5]', '$376,908.20'], ['State Grid Corporation of China', 'Electricity', '$530,009', '$8,192', '870,287', 'China', '', '[6]', '$609,004.85'], ['Vitol', 'Commodities', '$505,000', '$15,000', '1,560', 'Switzerland', '', '[7][8]', '$323,717,948.72'], ['China National Petroleum Corporation', 'Oil and gas', '$483,019', '$21,080', '1,087,049', 'China', '', '[9]', '$444,339.68'], ['China Petrochemical Corporation', 'Oil and gas', '$471,154', '$9,657', '527,487', 'China', '', '[10]', '$893,204.95'], ['ExxonMobil', 'Oil and gas', '$413,680', '$55,740', '63,000', 'United States', '', '[11]', '$6,566,349.21'], ['Apple', 'Electronics', '$394,328', '$99,803', '164,000', 'United States', '', '[12]', '$2,404,439.

In [94]:
# Here we come across another problem, for the 'Rank' header, it doesn't have any table data (td) tags for the data in it's
# column. This is a problem because when
# we extract our data into a dataframe we'll come across an error where we have too many columns and not enough data rows,
# so we have to remove the 'Rank' header. There are a multiple ways we can fix, but here are three
# 1. During our Data Clean up we can add and change the names of the columns, but if we had thousands of columns, this would
# take a relatively long time
# 2. Use a list comprehension to remove elements from a list based on their index
#    ex: headers = [x for i, x in enumerate(headers) if i != 0]
# 3. By combining slicing & concatenation we can remove an element from a list by index. The list is sliced into two parts
# the elements before the index and the elements after the index. Then, the two parts are concatenated to form a new 
#list without the element at index. But since it starts at the begining of the list we just need to slice 
#     ex: headers = headers[:2] + headers[3:]
#         ex: headers = headers[1:]

In [95]:
# Using List Comprehension

headers = [x for i, x in enumerate(headers) if i != 0]
print(headers)

['Name', 'Industry', 'Revenue', 'Profit', 'Employees', 'Headquarters[note 1]', 'State-owned', 'Ref.', 'Revenue per worker']


In [97]:
# Now we have our Table Headers and data for our rows so we create a DataFrame using them

df = pd.DataFrame(rows, columns=headers)

df

,Name,Industry,Revenue,Profit,Employees,Headquarters[note 1],State-owned,Ref.,Revenue per worker
0,Walmart,Retail,"$611,289","$11,680","2,100,000",United States,,[1],"$291,090.00"
1,Saudi Aramco,Oil and gas,"$603,651","$159,069","70,496",Saudi Arabia,,[4],"$8,562,911.37"
2,Amazon,Retail,"$574,785","$30,425","1,525,000",United States,,[5],"$376,908.20"
3,State Grid Corporation of China,Electricity,"$530,009","$8,192","870,287",China,,[6],"$609,004.85"
4,Vitol,Commodities,"$505,000","$15,000","1,560",Switzerland,,[7][8],"$323,717,948.72"
5,China National Petroleum Corporation,Oil and gas,"$483,019","$21,080","1,087,049",China,,[9],"$444,339.68"
6,China Petrochemical Corporation,Oil and gas,"$471,154","$9,657","527,487",China,,[10],"$893,204.95"
7,ExxonMobil,Oil and gas,"$413,680","$55,740","63,000",United States,,[11],"$6,566,349.21"
8,Apple,Electronics,"$394,328","$99,803","164,000",United States,,[12],"$2,404,439.02"
9,Shell,Oil and gas,"$386,201","$20,120","93,000",United Kingdom,,[13],"$4,152,698.92"


In [86]:
# After creating the DataFrame, you can save it to a CSV file for later use and manipulation

df.to_csv('largest_companies_by_revenue.csv', index=False)

# index=False: This argument ensures that the row indices are not saved to the CSV file.